# Binaural MUSDB18 Synthesis

In [ ]:
import os
import numpy as np
import soundfile as sf
from IPython.display import Audio

In [ ]:
# constants
HRIR_DIR = ""
SAMPLE_RATE = 44100

In [ ]:
# heads = ['D1', 'D2'] + [f'H{i}' for i in range(3, 21)]

In [ ]:
# get possible angles in front of the listener
angles = np.concatenate((np.arange(0, 91, 10), np.arange(270, 351, 10)))

In [ ]:
def make_binaural(y, angle):
    '''
    Turn a monophonic signal into a binaural 2-channel signal by
    convolving it with the left and right HRIRs for a given angle
    on the horizontal plane. The elevation for all locations is
    0 degrees.

    Parameters
    ----------
    y : monophonic input signal

    angle : target location of the source along the azimuth
        
    Returns
    -------
    y_binaural : 2-dimensional array with the
                 binaural left and right channels
    '''
    # load HRIR
    hrir_path = os.path.join(HRIR_DIR, f'azi_{angle},0_ele_0,0.wav')
    hrir, sr = sf.read(hrir_path)

    # convolve each channel with mono signal
    left = np.convolve(y, hrir[:, 0])
    right = np.convolve(y, hrir[:, 1])

    # combine into array
    binaural = np.vstack((left, right))
    
    return binaural